In [0]:
c_fail = -1
c_succ = 1
c_ff_num_partition=8
c_proc_name = 'fact-table-generation'


class CustomError(Exception):
    pass


from pyspark.sql import functions as f

from pyspark.sql.functions  import col


In [0]:

from pyspark.sql import SparkSession
spark = SparkSession. \
                builder. \
                appName("Aggregation"). \
                master("local[4]"). \
                getOrCreate()

In [0]:

      

class ut_store:
    '''place holder for  parameters'''

    sparksql_shuffle_partition_num = 4

    c_base_folder   = '/mnt/aota/'
    c_fold_sep      = '/'



    folder_raw= c_base_folder + 'raw/'
    folder_cln= c_base_folder + 'processed/'
    folder_intgr= c_base_folder + 'integrated/'
    folder_log =  c_base_folder + 'logs/'

    

    #Raw files name
    raw_file_plane = 'plane.csv'
    raw_file_flight ='2008.csv.bz2'
    raw_file_carrier = 'carrier.csv'
    raw_file_airport = 'airport.csv'

    #Processed/Clean file name
    cln_file_plane ='plane'
    cln_file_flight = 'flight'
    cln_file_carrier = 'carrier'
    cln_file_airport = 'airport'

    # dimension table data files
    file_dim_plane ='dim_plane'
    file_dim_date ='dim_date'
    file_dim_carrier='dim_carrier'
    file_dim_origin_dest = 'dim_origin_dest'

    # fact table data file
    file_fact_flight ='fact_flight'




In [0]:


from pyspark.sql.types import \
        StructType,\
        StringType, \
        IntegerType, \
        DateType, \
        TimestampType,\
        FloatType,\
        BooleanType

class ut_spark:
    """ a class of utitlies related to Spark, including tools to:
    1. get spark session
    2. read from/write to files
    3. translate atrribute list to spark schema



    """

    c_succ = True
    c_fail = False
    c_spark_sql_shuffle_partitions = ut_store.sparksql_shuffle_partition_num

    __spark =None

    @staticmethod
    def isFile(f_path):
        # Checking if a file exists with Pathlib
        file_path = Path(f_path)
        return file_path.is_file()

    @classmethod
    def log1_disp_error(cls, msg):
        print(msg)



    '''***********************************************************************

    Create Spark / Get spark session /Init

    ***********************************************************************'''

    @classmethod
    def creat_spark(cls):
        '''Create and return a spark session'''

        try:
            spark = SparkSession. \
                builder. \
                appName("Aggregation"). \
                master("local[4]"). \
                getOrCreate()

            spark.conf.set("spark.sql.shuffle.partitions", cls.c_spark_sql_shuffle_partitions)

            return spark

        except Exception as e:
            ut_log.log_exeption(e)
            return None


    @staticmethod
    def get_sparksession():
        if ut_spark.__spark==None:
            ut_spark.__spark=ut_spark.creat_spark()



        return ut_spark.__spark

    @classmethod
    def  initialise(cls):
        if cls.__spark == None:
         ut_spark.__spark = ut_spark.creat_spark()




    '''***********************************************************************
    
    Read from /Write to files
    
    ***********************************************************************'''

    @classmethod
    def df_read_from_csv(cls, file_path, schema=None):
        ''' Create a dataframe from a csv file,with header, with/without schema'''

        msg_fail = 'Spark-read-from-csv2 of {} failed --File does not exist!'.format((file_path))

        if cls.isFile(file_path) != True:
            msg_fail_on_file = 'Spark-read-from-csv1 of {} failed --File does not exist!'.format((file_path))
            ut_log.log_error(msg_fail_on_file)
            return None

        try:
            spark = cls.get_sparksession()
            if schema==None:
                  try:
                      return spark.read\
                       .option("header","true") \
                        .option("inferSchema","true") \
                        .csv(file_path)
                  except Exception as e:
                      ut_log.log_error(msg_fail)
                      ut_log.log_exeption(e)
                      return None
            else:
                  try:
                      return spark.read \
                          .option("header", "true") \
                          .schema(schema) \
                          .csv(file_path)
                  except:
                      ut_log.log_error(msg_fail)
                      return None
        except Exception as e:
            ut_log.log_error(msg_fail)
            ut_log.log_exeption(e)
            return None



    @classmethod
    def df_read_from_parquet(cls, file_path):

       try:
          spark = ut_spark.get_sparksession()
          df=  spark.read.parquet(file_path)
          return df
       except Exception as e:
           ut_log.log_exeption(e)
           return None



    @staticmethod
    def df_write_to_file(df, file_path, format="parquet"):

        msg_write_fail = "Writing file (file={}, format={}) failed.".format(file_path, format)

        if format not in ("csv","parquet","json"):
            ut_spark.log_disp_error("Invalide format. Action failed.")
            return ut_spark.c_fail

        try:
            df.write \
                .format(format) \
                .mode("overwrite") \
                .option("path", file_path) \
                .save()

            return ut_spark.c_succ
        except Exception as e:
            print(e)
            ut_log.log_error(msg_write_fail)
            ut_log.log_exeption(e)
            return ut_spark.c_fail


    '''***********************************************************************
    
    Schema  Translation
    
    ***********************************************************************'''


    @classmethod
    def list_to_schema(cls, fieldlist):
        def get_datatype(field_type):
            f_type = field_type[0:3]
            if f_type == 'str':
                return StringType()
            elif f_type == 'int':
                return IntegerType()
            elif f_type == "dat":
                return DateType()
            elif f_type == "boo":
                return BooleanType()
            elif f_type == "flo":
                return FloatType()
            else:
                return StringType()

        structSchema = StructType()

        for fieldTuple in fieldlist:
            structSchema.add(fieldTuple[0], get_datatype(fieldTuple[1]), True)

        return (structSchema)


    '''***********************************************************************
    
    Other tools
    ***********************************************************************'''

    @staticmethod
    def df_cr_tempview(df, table_name):
        try:
            df.createOrReplaceTempView(table_name)
        except Exception as e:
            ut_log.log_exeption(e)
            ut_log.log_warning(('Dataframe ({}) temp view creation failed.').format(df))



    @staticmethod
    def show_partition_count(df):
        df.groupBy(f.spark_partition_id()).count().show()

    @staticmethod
    def add_col(df, new_col, exp):
        '''Add a  monotonically increasing id column to a dataframe'''
        df1 = df.withColumn(new_col, exp)
        return df1

    @staticmethod
    def add_mono_incr_id(df, new_col):
        '''Add a  monotonically increasing id column to a dataframe'''
        df1 = df.withColumn(new_col, f.monotonically_increasing_id())
        return df1


    @staticmethod
    def df_dup_by_col(df, colname):
        return df.groupBy(col(colname)).agg(count(colname).alias("count")).filter(col("count") > 1)


    @staticmethod
    def get_partition_num(df):
        ''' Get number of partitions of a dataframe
        '''
        try:
            n = df.rdd.getNumPartitions()
            return n
        except Exception as e:
            ut_log.log_exeption(e)
            return None


if __name__ == "__main__":
    pass




In [0]:
class ut_log:
   def  log_info(msg):
      print(msg)
      
   def log_exeption(msg):
      print(msg)
      
                  
   def  log_proc_abort(c_proc_name):
     pass
    
   def  log_proc_metric(name):
      pass
      


In [0]:
import time
import contextlib

from datetime import  datetime


from pathlib import Path

class ut_base:
    const_succ = 1
    const_fail = -1
    date_format = "%m/%d/%Y-%H:%M:%S"
    F_FAIL = -1

    '''****************************************************************
            Time/Date related methods
    ********************************************************************'''

    @staticmethod
    def curr_time():
      return datetime.now()

    @staticmethod
    def curr_date():
      return datetime.today()

    @staticmethod
    def curr_time_str():
      ''' Get a string out of current timestamp. Format: 20220321_120301'''
      return ut_base.time_to_str(datetime.now(),"%Y%m%d_%H%M%S")

    @staticmethod
    def curr_time_fmt():
      ''' Get a string out of current timestamp. Format: 2022-03-21_12:03:01'''

      return ut_base.time_to_str(datetime.now(),"%Y-%m-%d_%H:%M:%S")


    @staticmethod
    def time_to_str(p_datetime, p_dt_format=None):
      '''To conver a timestamp to a string.  Format can vary or be empty.'''
      if p_dt_format ==None:
         p_dt_format = ut_base.date_format

      return p_datetime.strftime(p_dt_format)

    @staticmethod
    def str_to_time(p_dt_string, p_dt_format=None):
      '''To convert a string to a timestamp .  Format can vary or be empty.'''
      if p_dt_format ==None:
         p_dt_format =ut_base.date_format
      return datetime.strptime(p_dt_string, p_dt_format)



    @staticmethod
    @contextlib.contextmanager
    def timer():
      """Time the execution of a context block.

      Yields:
        None
      """
      start = time.time()
      # Send control back to the context block
      yield
      end = time.time()
      #print('Elapsed: {:.2f}s'.format(end - start))
      print('Elapsed:%6f'%(end - start))


    '''****************************************************************
            Methods to pring dict, list and check file path
    ********************************************************************'''

    @staticmethod
    def isFile(f_path):
      # Checking if a file exists with Pathlib
      file_path = Path(f_path)
      return file_path.is_file()


    def print_dict(p_dict):
      line1 = ''
      for key in p_dict:
        line1 = line1 + '{}:{},'.format(key, p_dict[key])

      print(line1)

    def print_dict_val(p_dict:dict):
      line1 = ''
      for key in p_dict:
        line1 = line1 + '{},'.format(p_dict[key])
      print(line1)



    @staticmethod
    def print_list(list:list):
      '''Print a list object'''
      [print(l) for l in list]


    @staticmethod
    def print_dict_v(p_dict):
          '''Print  all the values of a dict object'''
          line1=''
          for key in p_dict:
              line1=line1+'{},'.format(p_dict[key])
          print(line1)

    def print_dict(p_dict):
          '''Print out all the keys of a dict object '''
          line1=''
          for key in p_dict:
              line1=line1+'{}:{},'.format(key,p_dict[key])

In [0]:
def gen_date_key( year, month, day):
        sep = ''
        return str(year) + sep + str(month).rjust(2, '0') + sep + str(day).rjust(2, '0')
    
def gen_orig_dest_key(origin,dest):
        sep = '_'
        return origin + sep + dest



        c_proc_name= 'fact-table-generation'
        udf_gen_date_key = f.udf(gen_date_key, returnType=f.StringType())
        udf_gen_orig_dest_key = f.udf(gen_orig_dest_key, returnType=f.StringType())

        rfp=ut_store.folder_cln + ut_store.cln_file_flight 
        #+'/year=1998'
        print(rfp)

        df_fl = ut_spark.df_read_from_parquet(rfp)  


        # Adding date_key for buildinng dim_date later
        df_fl1 = df_fl.withColumn("date_id", udf_gen_date_key("Year", "Month", "DayofMonth"))

        # Adding orig_dest_key for buildinng dim_orign_dest
        df_fl2 = df_fl1.withColumn("orig_dest_key", udf_gen_orig_dest_key("Dest", "Origin"))

        df_fl_base = ut_spark.add_mono_incr_id(df_fl2, "seq_id")
        df_fl_base.groupBy("year").count().show()
        
        return df_fl_base
        

In [0]:

        c_proc_name= 'fact-table-generation'
        udf_gen_date_key = f.udf(gen_date_key, returnType=f.StringType())
        udf_gen_orig_dest_key = f.udf(gen_orig_dest_key, returnType=f.StringType())

        rfp=ut_store.folder_cln + ut_store.cln_file_flight 
        #+'/year=1998'
        print(rfp)

        df_fl = spark.read.parquet(rfp)  


       

/mnt/aota/processed/flight


In [0]:

        # Adding date_key for buildinng dim_date later
        df_fl1 = df_fl.withColumn("date_id", udf_gen_date_key("Year", "Month", "DayofMonth"))

        # Adding orig_dest_key for buildinng dim_orign_dest
        df_fl2 = df_fl1.withColumn("orig_dest_key", udf_gen_orig_dest_key("Dest", "Origin"))



In [0]:
        df_fl_base = ut_spark.add_mono_incr_id(df_fl2, "seq_id")
        df_fl_base.groupBy("year").count().show()

+----+-------+
|year|  count|
+----+-------+
|2006|7003802|
|2004|6987729|
|2005|6992838|
|2003|6375690|
|2002|5197860|
|2001|5723673|
|1999|5360018|
|2000|5481303|
|1997|5301999|
|1998|5227051|
+----+-------+



In [0]:
''' **************************************************
              Create dimension table -- dim_date
   *****************************************************'''
df_date = df_fl_base.select("date_id", "Year", "Month", "DayofMonth", "DayOfWeek").distinct()
res = ut_spark.df_write_to_file(df_date, ut_store.folder_intgr + "dim_date", format="parquet")
if res ==ut_spark.c_fail:
    raise CustomError
ut_log.log_info("dim_date created.")

dim_date created.


In [0]:
df_date.count()

Out[61]: 3652

In [0]:
''' **************************************************
  Create dimension table -- dim_orig_dest
*****************************************************'''


df_od1 = df_fl_base.select("orig_dest_key", "origin", "dest", "distance").distinct()
df_ap = ut_spark.df_read_from_parquet(ut_store.folder_cln + 'airport')

df_od2 = df_od1.join(df_ap, df_od1["Origin"] == df_ap["iata"]). \
    select(col("orig_dest_key")
           , col("origin")
           , col("dest")
           , col("distance")
           , col("airport").alias("orig_airport")
           , col("city").alias("orig_city")
           , col("state").alias("orig_state")
           , col("country").alias("orig_country")
           )

df_od3 = df_od2.join(df_ap, df_od1["Dest"] == df_ap["iata"]). \
    select(col("orig_dest_key").alias("orig_dest_id")
           , col("origin")
           , col("dest")
           , col("distance")
           , col("orig_airport")
           , col("orig_city")
           , col("orig_state")
           , col("orig_country")
           , col("airport").alias("dest_airport")
           , col("city").alias("dest_city")
           , col("state").alias("dest_state")
           , col("country").alias("dest_country")
           )

res = ut_spark.df_write_to_file(df_od3, ut_store.folder_intgr + "dim_origin_dest", format="parquet")
if res ==ut_spark.c_fail:
    raise CustomError
ut_log.log_info("dim_orig_dest created.")


dim_orig_dest created.


In [0]:


''' **************************************************
  Create fact table -- fact_flight
*****************************************************'''
df_fact_flt =df_fl_base.select\
        (col("year")
        , col("month")
        , col("seq_id")
        , col("date_id")
        , col("orig_dest_key").alias("orig_dest_id")
        , col("flightNum")
        , col("uniqueCarrier").alias("carrier_id")
        , col("tailNum").alias("plane_id")
        , col("DepTime")
        , col("CRSDepTime").alias("scheduled_deptime")
        , col("ArrTime")
        , col("CRSArrTime").alias("scheduled_arrtime")
        , col("ArrDelay")
        , col("DepDelay")
        )

ut_log.log_info('df_fact_flt created.')
df_fact_flt.groupBy("year").count().show()
df_fact_flt.printSchema()

# partition the fact dataset by "seq_id" (using seq_id to average the size of each partition)
df_fact_flt_repart=df_fact_flt.repartition(c_ff_num_partition, "seq_id")
ut_spark.show_partition_count(df_fact_flt_repart)

'''-------------------------------------------------------
Write fact table to storing location ("integeratio" layer)
--------------------------------------------------------'''

#get file path
wf_path = ut_store.folder_intgr + ut_store.file_fact_flight

#Process by year: collect year information to df2
df2 = df_fact_flt_repart.groupBy("year").count().orderBy("year")

dataCollect = df2.collect()
for row in dataCollect:
    year = row['year']

    df3 = df_fact_flt_repart.filter(f.col("year") == year)
    print('year={}:count={}'.format(year, df3.count()))
    #df3.write.format("parquet") \
    df3.write.format("parquet") \
        .mode("append") \
        .partitionBy("year") \
        .option("path", wf_path) \
        .save()

ut_log.log_info("fact_flight created.")


df_fact_flt created.
+----+-------+
|year|  count|
+----+-------+
|2006|7003802|
|2004|6987729|
|2005|6992838|
|2003|6375690|
|2002|5197860|
|2001|5723673|
|1999|5360018|
|2000|5481303|
|1997|5301999|
|1998|5227051|
+----+-------+

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- seq_id: long (nullable = false)
 |-- date_id: string (nullable = true)
 |-- orig_dest_id: string (nullable = true)
 |-- flightNum: string (nullable = true)
 |-- carrier_id: string (nullable = true)
 |-- plane_id: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- scheduled_deptime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- scheduled_arrtime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)

+--------------------+-------+
|SPARK_PARTITION_ID()|  count|
+--------------------+-------+
|                   0|7454912|
|                   1|7460719|
|                   2|7458282|
